In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import f1_score,roc_auc_score,mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from matplotlib import pyplot

In [2]:
filepath = r'/content/drive/MyDrive/Untitled folder/salary_satisfaction.csv'
df =  pd.read_csv(filepath,engine='python')

In [3]:
df.head()

,id,Target_Salary,Target_Satisfied,LOAN_AMT,Business Title,Civil Service Title,Division/Work Unit,Job Description,Minimum Qual Requirements,Preferred Skills,Additional Information,To Apply,Hours/Shift,Residency Requirement,Posting Date,Posting Updated,Process Date,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,YearsCodingProf,HopeFiveYears,JobSearchStatus,LastNewJob,UpdateCV,CommunicationTools,TimeFullyProductive,EducationTypes,SelfTaughtTypes,TimeAfterBootcamp,HackathonReasons,AgreeDisagree1,AgreeDisagree2,AgreeDisagree3,LanguageWorkedWith,LanguageDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,...,FrameworkDesireNextYear,IDE,OperatingSystem,Methodology,VersionControl,CheckInCode,AIDangerous,AIInteresting,AIResponsible,AIFuture,Age,term,loan_status,pymnt_plan,orignal_website_directory,purpose,title,zip_code,addr_state,dti,earliest_cr_line,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,Time,emp_length,your_favoritearticle_today,url,Email,homeaddress,latitude,longitude,PHONE,officeaddress,website,dateAdded,previousaddress,phones,CrimeTime
0,68407277,42405.00,0,"$3,600.00",Account Manager,CONTRACT REVIEWER (OFFICE OF L,Strategy & Analytics,Division of Economic & Financial Opportunity (...,1.\tA baccalaureate degree from an accredited ...,•\tExcellent interpersonal and organizational ...,"Salary range for this position is: $42,405 - $...",NaN,NaN,New York City residency is generally required ...,2011-06-24T00:00:00.000,2011-06-24T00:00:00.000,2019-12-17T00:00:00.000,"Bachelor�s degree (BA, BS, B.Eng., etc.)",Mathematics or statistics,20 to 99 employees,Full-stack developer,3-5 years,3-5 years,Working as a founder or co-founder of my own c...,"I�m not actively looking, but I am open to new...",Less than a year ago,My job status or other personal status changed,Slack,One to three months,"Taught yourself a new language, framework, or ...",The official documentation and/or standards fo...,NaN,To build my professional network,Strongly agree,Strongly agree,Neither Agree nor Disagree,JavaScript;Python;HTML;CSS,JavaScript;Python;HTML;CSS,Redis;SQL Server;MySQL;PostgreSQL;Amazon RDS/A...,Redis;SQL Server;MySQL;PostgreSQL;Amazon RDS/A...,...,Django;React,Komodo;Vim;Visual Studio Code,Linux-based,Agile;Scrum,Git,Multiple times per day,Artificial intelligence surpassing human intel...,Algorithms making important decisions,The developers or the people creating the AI,I'm excited about the possibilities more than ...,25 - 34 years old,36 months,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,190xx,PA,5.91,3-Aug,19-Jan,122.67,NaN,19-Mar,3/22/2016 9:51,< 1 year,https://www.reuters.com/article/us-tesla-crash...,nobell.it/70ffb52d079109dca5664cce6f317373782/...,mstephenson@fernandez.com,"346 W Magnolia Ave Auburn, AL 36832 US",32.606813,-85.487328,(91) 555 22 82,"PO Box 4653, Stockton, California, 95204",http://www.valleybrew.com/,2016-04-22T02:47:48Z,1045 San Pablo Ave,(510) 528-2375,23:30:00
1,68355089,60740.00,1,"$24,700.00","EXECUTIVE DIRECTOR, BUSINESS DEVELOPMENT",ADMINISTRATIVE BUSINESS PROMOT,Tech Talent Pipeline,The New York City Department of Small Business...,1. A baccalaureate degree from an accredited c...,NaN,NaN,"In addition to applying through this website, ...",NaN,New York City residency is generally required ...,2012-01-26T00:00:00.000,2012-01-26T00:00:00.000,2019-12-17T00:00:00.000,"Bachelor�s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...","10,000 or more employees",Database administrator;DevOps specialist;Full-...,30 or more years,18-20 years,Working in a different or more specialized tec...,I am actively looking for a job,More than 4 years ago,I saw an employer�s advertisement,Confluence;Office / productivity suite (Micros...,One to three months,"Taught yourself a new language, framework, or ...",The official documentation and/or standards fo...,NaN,NaN,Agree,Agree,Neither Agree nor Disagree,JavaScri

In [4]:
df_n=df

In [5]:
'''Size of the DataSet'''
df_n.shape

(2946, 83)

In [6]:
''' Column details'''
df_n.columns

Index(['id', 'Target_Salary', 'Target_Satisfied', 'LOAN_AMT', 'Business Title',
       'Civil Service Title', 'Division/Work Unit', 'Job Description',
       'Minimum Qual Requirements', 'Preferred Skills',
       'Additional Information', 'To Apply', 'Hours/Shift',
       'Residency Requirement', 'Posting Date', 'Posting Updated',
       'Process Date', 'FormalEducation', 'UndergradMajor', 'CompanySize',
       'DevType', 'YearsCoding', 'YearsCodingProf', 'HopeFiveYears',
       'JobSearchStatus', 'LastNewJob', 'UpdateCV', 'CommunicationTools',
       'TimeFullyProductive', 'EducationTypes', 'SelfTaughtTypes',
       'TimeAfterBootcamp', 'HackathonReasons', 'AgreeDisagree1',
       'AgreeDisagree2', 'AgreeDisagree3', 'LanguageWorkedWith',
       'LanguageDesireNextYear', 'DatabaseWorkedWith',
       'DatabaseDesireNextYear', 'PlatformWorkedWith',
       'PlatformDesireNextYear', 'FrameworkWorkedWith',
       'FrameworkDesireNextYear', 'IDE', 'OperatingSystem', 'Methodology',
       'V

# Data Cleaning

In [7]:
''' removing all the space,punctuations,\t,numeric valus'''

def clean_catagorical_data(x):
  pattern=re.compile(r'[-\s\t\d,(\)�:#.\/;"•]')
  x = x .str.replace(pattern,'')
  x = x.str.replace('&','and')
  x=x.replace(np.nan, 'Nan')
  x= x.str.lower()
  
  return x


In [8]:
def Transform_Categorical_Features(Data,features):
  for feature in features:
      Data[feature] = clean_catagorical_data(Data[feature])
  return Data

In [9]:
df_n=Transform_Categorical_Features(df_n,features=['Business Title','Civil Service Title','Division/Work Unit','Job Description','Minimum Qual Requirements','Preferred Skills','Additional Information','To Apply','Residency Requirement','FormalEducation','UndergradMajor','HopeFiveYears','JobSearchStatus','LastNewJob','UpdateCV','CommunicationTools','EducationTypes','SelfTaughtTypes','TimeAfterBootcamp','HackathonReasons','CheckInCode','AIDangerous','AIInteresting','AIResponsible','AIFuture','loan_status','purpose','title','addr_state'])

In [10]:
p=re.compile('[,$]')
df_n['LOAN_AMT']=df_n['LOAN_AMT'].str.replace(p,'').astype(float)


In [11]:
''' Convert Posting Date to date time format'''

df_n['Posting Date']=pd.to_datetime(df_n['Posting Date'])
df_n['Posting_Date_day']=np.uint8(df_n['Posting Date'].dt.day)
df_n['Posting_Date_month']=np.uint8(df_n['Posting Date'].dt.month)
df_n['Posting_Date_year']=np.uint8(df_n['Posting Date'].dt.year)


In [12]:
''' Drop Posting Date after converting to Date time format'''
df_n.drop(columns=['Posting Date'],inplace=True,axis=1)

In [13]:
df_n.shape

(2946, 85)

In [14]:
''' Convert Posting Updated to date time format'''

df_n['Posting Updated']=pd.to_datetime(df_n['Posting Updated'])
df_n['Posting_Updated_day']=np.uint8(df_n['Posting Updated'].dt.day)
df_n['Posting_Updated_month']=np.uint8(df_n['Posting Updated'].dt.month)
df_n['Posting_Updated_year']=np.uint8(df_n['Posting Updated'].dt.year)


In [15]:
''' Drop Posting Updated after converting to Date time format'''
df_n.drop(columns=['Posting Updated'],inplace=True,axis=1)

In [16]:
df_n.shape

(2946, 87)

In [17]:
''' Convert Process Date to date time format'''

df_n['Process Date']=pd.to_datetime(df_n['Process Date'])
df_n['Process_Date_day']=np.uint8(df_n['Process Date'].dt.day)
df_n['Process_Date_month']=np.uint8(df_n['Process Date'].dt.month)
df_n['Process_Date_year']=np.uint8(df_n['Process Date'].dt.year)

In [18]:
''' Drop Process Date after converting to Date time format'''
df_n.drop(columns=['Process Date'],inplace=True,axis=1)

In [19]:
df_n.shape

(2946, 89)

In [20]:
''' removing all the space,punctuations,Nan'''

def clean_catagorical_data1(x):
  pattern=re.compile(r'[\s,(\)\/"]')
  x = x .str.replace(pattern,'')
  x = x.str.replace('-','_')
  x = x.str.replace('.','_')
  x=x.replace(np.nan, 'Nan')
  x= x.str.lower()
  return x


In [21]:
def Transform_Categorical_Features1(Data,features):
  for feature in features:
      Data[feature] = clean_catagorical_data1(Data[feature])
  return Data

In [22]:
df_n=Transform_Categorical_Features1(df_n,features=['YearsCoding','CompanySize','YearsCodingProf','DevType','TimeFullyProductive','LanguageWorkedWith','LanguageDesireNextYear','DatabaseWorkedWith','DatabaseDesireNextYear','PlatformWorkedWith','PlatformDesireNextYear','FrameworkWorkedWith','FrameworkDesireNextYear','IDE','Methodology','VersionControl','CheckInCode','AIDangerous','AIInteresting','AIResponsible','AIResponsible','AIFuture','Age','term','loan_status','title','AgreeDisagree1','AgreeDisagree2','AgreeDisagree3','earliest_cr_line','last_pymnt_d','next_pymnt_d','last_credit_pull_d','emp_length','previousaddress','phones'])

In [23]:
''' The value in all the cells are same in 'pymnt_plan' ,so it does not contain any information which can help to analysis the data. 
   that is why i droped the column '''

df_n.drop(columns=['pymnt_plan'],inplace=True,axis=1)

In [24]:
p=re.compile('[a-zA-Z/?.:_=]')
df_n['orignal_website_directory']=df_n['orignal_website_directory'].str.replace(p,'')

In [25]:
''' Convert Time to date time format'''

df_n['Time']=pd.to_datetime(df_n['Time'])
df_n['Timee_day']=np.uint8(df_n['Time'].dt.day)
df_n['Time_month']=np.uint8(df_n['Time'].dt.month)
df_n['Time_year']=np.uint8(df_n['Time'].dt.year)
df_n['Time_hour']=np.uint8(df_n['Time'].dt.hour)
df_n['Time_min']=np.uint8(df_n['Time'].dt.minute)

In [26]:
''' Drop Time after converting to Date time format'''
df_n.drop(columns=['Time'],inplace=True,axis=1)

In [27]:
p1=re.compile('[/.:\s-]')
df_n['website']=df_n['website'].str.replace(p1,'')
df_n['website']=df_n['website'].str.replace("http",'')
df_n['website']=df_n['website'].str.replace("www",'')
df_n['website']=df_n['website'].replace(np.nan, 'Nan')
df_n['website']=df_n['website'].str.lower()

In [28]:
''' Convert dateAdded to date time format'''

df_n['dateAdded']=pd.to_datetime(df_n['dateAdded'])
df_n['dateAdded_day']=np.uint8(df_n['dateAdded'].dt.day)
df_n['dateAdded_month']=np.uint8(df_n['dateAdded'].dt.month)
df_n['dateAdded_year']=np.uint8(df_n['dateAdded'].dt.year)
df_n['dateAdded_hour']=np.uint8(df_n['dateAdded'].dt.hour)
df_n['dateAdded_min']=np.uint8(df_n['dateAdded'].dt.minute)
df_n['dateAdded_sec']=np.uint8(df_n['dateAdded'].dt.second)

In [29]:
''' Drop dateAdded after converting to Date time format'''
df_n.drop(columns=['dateAdded'],inplace=True,axis=1)

In [30]:
''' Convert CrimeTime to date time format'''

df_n['CrimeTime']=pd.to_datetime(df_n['CrimeTime'])
df_n['CrimeTime_hour']=np.uint8(df_n['CrimeTime'].dt.hour)
df_n['CrimeTime_min']=np.uint8(df_n['CrimeTime'].dt.minute)

In [31]:
''' Drop CrimeTime after converting to Date time format'''
df_n.drop(columns=['CrimeTime'],inplace=True,axis=1)

In [32]:
p=re.compile('[\s:;.#-,()/@â€“]')
df_n['Hours/Shift']=df_n['Hours/Shift'].str.replace(p,'')
df_n['Hours/Shift']=df_n['Hours/Shift'].str.replace('hrs','hours')
df_n['Hours/Shift']=df_n['Hours/Shift'].str.replace('x','')
df_n['Hours/Shift']=df_n['Hours/Shift'].replace(np.nan,'Nan')
df_n['Hours/Shift']=df_n['Hours/Shift'].str.lower()


In [33]:
df_n['latitude']=np.uint8(df_n['latitude'])
df_n['longitude']=np.uint8(df_n['longitude'])

In [34]:
'''Convert 'latitude','longitude' to cartesian coordinate'''

def cordinate(df):
  df['x-co'] = pd.Series(np.uint8([np.cos(lat) * np.cos(lon) for lat,lon in df[['latitude','longitude']].values]))
  df['y-co'] = pd.Series(np.uint8([np.cos(lat) * np.sin(lon) for lat,lon in df[['latitude','longitude']].values]))
  df['z-co'] = pd.Series(np.uint8([np.sin(lat) for lat in df['latitude'].values]))
  return df

In [35]:
df_n = cordinate(df_n)

In [36]:
df_n['x-co'].value_counts()

0    2629
1     317
Name: x-co, dtype: int64

In [37]:
'''Drop some features which are irreevant to do this analysis'''

df_n.drop(columns=['your_favoritearticle_today','url','PHONE','id','latitude','longitude','homeaddress','officeaddress','zip_code'],inplace=True,axis=1)

In [38]:
df_n['Email']=df_n['Email'].str.replace('@',' ')
df_n['Email']=df_n['Email'].str.replace('-','_')
df_n['Email']=df_n['Email'].str.replace('.com','')
df_n['Email']=df_n['Email'].replace(np.nan, 'Nan')
df_n['Email']=df_n['Email'].str.lower()

In [39]:
df_n.to_csv('/content/drive/MyDrive/Untitled folder/salary_satisfection_pre_n.csv',index=False)

In [40]:
df_n.shape

(2946, 92)

# Clssification Problem

## Train Test Split

In [77]:
df_n1 =  pd.read_csv('/content/drive/MyDrive/Untitled folder/salary_satisfection_pre_n.csv')

In [78]:
df_n1.shape

(2946, 92)

In [79]:
''' Creat target column for classification model'''
y_Target_Satisfied=df_n1['Target_Satisfied']
df_n1.drop(columns=['Target_Satisfied'],inplace=True,axis=1)


In [80]:
''' Train test split randomly'''
x_train, x_test, y_train_class, y_test_class = train_test_split(df_n1, y_Target_Satisfied, test_size=0.20, random_state=42)

In [81]:
print(x_train.shape,x_test.shape)
print(y_train_class.shape,y_test_class.shape)

(2356, 91) (590, 91)
(2356,) (590,)


## Standardization of numerical data

In [82]:
def standardscalertransform(data,data1,features):
  for feature in features:
    scaler = StandardScaler()
    scaler.fit(data[feature].values.reshape(-1,1))
    data[feature] =scaler.transform(data[feature].values.reshape(-1,1))
    data1[feature] =scaler.transform(data1[feature].values.reshape(-1,1))
  return data,data1

In [83]:
x_train,x_test=standardscalertransform(x_train,x_test,features=['Target_Salary','LOAN_AMT','dti','last_pymnt_amnt','x-co','y-co','z-co'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

## Text Preprocessing (One hot encoding)

In [84]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack,csr_matrix
from tqdm import tqdm
def Vectorizer(x_train,x_test,features):
  count = 0
  for feature in features:
    vect = CountVectorizer(ngram_range=(1,1))
    vect.fit(x_train[feature].values.astype('U'))
    x_train_vect = vect.transform(x_train[feature].values.astype('U'))
    x_test_vect = vect.transform(x_test[feature].values.astype('U'))
    if count == 0:
      x_train_df=x_train_vect
      x_test_df=x_test_vect
      print(x_train_df.shape,x_test_df.shape)
    else:
      x_train_df=hstack((x_train_df,x_train_vect,)).tocsr()
      x_test_df=hstack((x_test_df,x_test_vect,)).tocsr()
      print(x_train_df.shape,x_test_df.shape)
    count+=1 
  return x_train_df,x_test_df

In [85]:
columns_ohe=x_train.columns.tolist()
print('Total Column',len(columns_ohe))
element_remove=['Target_Salary','LOAN_AMT','dti','last_pymnt_amnt','x-co','y-co','z-co','Posting_Date_day','Posting_Date_month','Posting_Date_year','Posting_Updated_day','Posting_Updated_month','Posting_Updated_year','Process_Date_day','Process_Date_month','Process_Date_year','Timee_day','Time_month','Time_year','Time_hour','Time_min',
                   'dateAdded_day','dateAdded_month','dateAdded_year','dateAdded_hour','dateAdded_min','dateAdded_sec','CrimeTime_hour','CrimeTime_min']
print('Total Column',len(element_remove))
for i in element_remove:
  columns_ohe.remove(i)
print('Column for One Hot Encodin',len(columns_ohe))




Total Column 91
Total Column 29
Column for One Hot Encodin 62


In [86]:
x_train_df,x_test_df = Vectorizer(x_train,x_test,columns_ohe)                                                

(2356, 1115) (590, 1115)
(2356, 1416) (590, 1416)
(2356, 2058) (590, 2058)
(2356, 7041) (590, 7041)
(2356, 7601) (590, 7601)
(2356, 9389) (590, 9389)
(2356, 10094) (590, 10094)
(2356, 10644) (590, 10644)
(2356, 10794) (590, 10794)
(2356, 10831) (590, 10831)
(2356, 10841) (590, 10841)
(2356, 10854) (590, 10854)
(2356, 10862) (590, 10862)
(2356, 10883) (590, 10883)
(2356, 10895) (590, 10895)
(2356, 10907) (590, 10907)
(2356, 10916) (590, 10916)
(2356, 10920) (590, 10920)
(2356, 10925) (590, 10925)
(2356, 10934) (590, 10934)
(2356, 11249) (590, 11249)
(2356, 11256) (590, 11256)
(2356, 11497) (590, 11497)
(2356, 11704) (590, 11704)
(2356, 11713) (590, 11713)
(2356, 11808) (590, 11808)
(2356, 11814) (590, 11814)
(2356, 11820) (590, 11820)
(2356, 11826) (590, 11826)
(2356, 11860) (590, 11860)
(2356, 11894) (590, 11894)
(2356, 11916) (590, 11916)
(2356, 11938) (590, 11938)
(2356, 11965) (590, 11965)
(2356, 11992) (590, 11992)
(2356, 12005) (590, 12005)
(2356, 12018) (590, 12018)
(2356, 12041)

In [87]:
x_train_df.shape

(2356, 18281)

In [88]:
len(element_remove)

29

In [89]:
''' Concat the nontext features with the x_train_df that we have got after doing one hot encoding of text features'''

def add_features(x_train,x_test,x_train_df,x_test_df,features):
  count = 0
  for feature in features:
    if count == 0:
      x_train_df = np.hstack((x_train_df.toarray(),x_train[feature].values.reshape(-1,1)))
      x_test_df = np.hstack((x_test_df.toarray(),x_test[feature].values.reshape(-1,1)))
    else:
      x_train_df = np.hstack((x_train_df,x_train[feature].values.reshape(-1,1)))
      x_test_df = np.hstack((x_test_df,x_test[feature].values.reshape(-1,1)))
    count +=1
  return x_train_df,x_test_df

In [90]:
x_tr,x_te=add_features(x_train,x_test,x_train_df,x_test_df,element_remove)

In [91]:
x_tr.shape,x_te.shape

((2356, 18310), (590, 18310))

# Model

# Feature Importance

In [ ]:
model = RandomForestClassifier()
model.fit(x_tr, y_train_class)
importance = model.feature_importances_

In [ ]:
importance_list = importance.tolist()

In [ ]:
index_list=list()
feature_importance_val=list()
for index,value in enumerate(importance_list):
  index_list.append(index)
  feature_importance_val.append(value)

In [ ]:
feature_df = pd.DataFrame.from_dict({'Feauture_index':index_list,'Feature_val':feature_importance_val})

In [ ]:
feature_df.sort_values(by=['Feature_val'], ascending=False)

,Feauture_index,Feature_val
18284,18284,0.004044
18282,18282,0.003766
18283,18283,0.003404
18288,18288,0.002860
18281,18281,0.002559
...,...,...
1888,1888,0.000000
8846,8846,0.000000
8850,8850,0.000000
8851,8851,0.000000


In [ ]:
feature_df_sort = feature_df[feature_df.Feature_val != 0].sort_values(by=['Feature_val'], ascending=False)

In [ ]:
important_feature_list = feature_df_sort['Feauture_index'].tolist()

In [ ]:
len(important_feature_list)

7519

In [ ]:
x_tr_important=x_tr[:,important_feature_list]
x_te_important=x_te[:,important_feature_list]

In [ ]:
x_tr_important.shape

(2356, 7519)

## Hyperparameter tuning

In [111]:
clf = RandomForestClassifier()
grid_params = {'n_estimators': [10, 50, 100, 200], 'max_depth':[1, 5, 10, 20,30,50]}
gs = GridSearchCV(clf, grid_params, cv=5, scoring='f1',n_jobs=-1,return_train_score=True)
gs.fit(x_tr_important, y_train_class)


results = pd.DataFrame.from_dict(gs.cv_results_)
print(results.columns)

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_max_depth', 'param_n_estimators', 'params', 'split0_test_score',
       'split1_test_score', 'split2_test_score', 'split3_test_score',
       'split4_test_score', 'mean_test_score', 'std_test_score',
       'rank_test_score', 'split0_train_score', 'split1_train_score',
       'split2_train_score', 'split3_train_score', 'split4_train_score',
       'mean_train_score', 'std_train_score'],
      dtype='object')


In [753]:
best_param_clas=gs.best_params_
best_param_clas

{'max_depth': 50, 'n_estimators': 10}

## Final model 

In [127]:
neigh = RandomForestClassifier(max_depth=50,n_estimators=10,random_state=0)
neigh.fit(x_tr_important, y_train_class)

y_train_pred = neigh.predict(x_tr_important)
y_test_pred = neigh.predict(x_te_important)

f1_score_train=f1_score(y_train_class, y_train_pred,average='weighted')
f1_score_test=f1_score(y_test_class, y_test_pred,average='weighted')

In [209]:
print('F1 score for train',round(f1_score_train1,4))
print('F1 score for test',round(f1_score_test1,4))

F1 score for train 0.7869
F1 score for test 0.7252


# Regression Problem

## Train Teat Split

In [180]:
df_n1 =  pd.read_csv('/content/drive/MyDrive/Untitled folder/salary_satisfection_pre_n.csv')

In [181]:
df_n1.shape

(2946, 92)

In [182]:
''' Creat target column for classification model'''
y_Target_Salary=df_n1['Target_Salary']
df_n1.drop(columns=['Target_Salary','Target_Satisfied'],inplace=True,axis=1)


In [183]:
''' Train test split randomly'''
x_train_r, x_test_r, y_train_reg, y_test_reg = train_test_split(df_n1, y_Target_Salary, test_size=0.20, random_state=42)

In [184]:
print(x_train_r.shape,x_test_r.shape)
print(y_train_reg.shape,y_test_reg.shape)

(2356, 90) (590, 90)
(2356,) (590,)


## Standardization of numerical data

In [185]:
def standardscalertransform(data,data1,features):
  for feature in features:
    scaler = StandardScaler()
    scaler.fit(data[feature].values.reshape(-1,1))
    data[feature] =scaler.transform(data[feature].values.reshape(-1,1))
    data1[feature] =scaler.transform(data1[feature].values.reshape(-1,1))
  return data,data1

In [186]:
x_train_r,x_test_r=standardscalertransform(x_train_r,x_test_r,features=['LOAN_AMT','dti','last_pymnt_amnt','x-co','y-co','z-co'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

## Text Preprocessing (One hot encoding)

In [187]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack,csr_matrix
from tqdm import tqdm
def Vectorizer(x_train,x_test,features):
  count = 0
  for feature in features:
    vect = CountVectorizer(ngram_range=(1,1))
    vect.fit(x_train[feature].values.astype('U'))
    x_train_vect = vect.transform(x_train[feature].values.astype('U'))
    x_test_vect = vect.transform(x_test[feature].values.astype('U'))
    if count == 0:
      x_train_df=x_train_vect
      x_test_df=x_test_vect
      print(x_train_df.shape,x_test_df.shape)
    else:
      x_train_df=hstack((x_train_df,x_train_vect,)).tocsr()
      x_test_df=hstack((x_test_df,x_test_vect,)).tocsr()
      print(x_train_df.shape,x_test_df.shape)
    count+=1 
  return x_train_df,x_test_df

In [188]:
columns_ohe=x_train_r.columns.tolist()
print('Total Column',len(columns_ohe))
element_remove=['LOAN_AMT','dti','last_pymnt_amnt','x-co','y-co','z-co','Posting_Date_day','Posting_Date_month','Posting_Date_year','Posting_Updated_day','Posting_Updated_month','Posting_Updated_year','Process_Date_day','Process_Date_month','Process_Date_year','Timee_day','Time_month','Time_year','Time_hour','Time_min',
                   'dateAdded_day','dateAdded_month','dateAdded_year','dateAdded_hour','dateAdded_min','dateAdded_sec','CrimeTime_hour','CrimeTime_min']
print('Total Column',len(element_remove))
for i in element_remove:
  columns_ohe.remove(i)
print('Column for One Hot Encodin',len(columns_ohe))




Total Column 90
Total Column 28
Column for One Hot Encodin 62


In [189]:
x_train_df,x_test_df = Vectorizer(x_train_r,x_test_r,columns_ohe)                                                

(2356, 1115) (590, 1115)
(2356, 1416) (590, 1416)
(2356, 2058) (590, 2058)
(2356, 7041) (590, 7041)
(2356, 7601) (590, 7601)
(2356, 9389) (590, 9389)
(2356, 10094) (590, 10094)
(2356, 10644) (590, 10644)
(2356, 10794) (590, 10794)
(2356, 10831) (590, 10831)
(2356, 10841) (590, 10841)
(2356, 10854) (590, 10854)
(2356, 10862) (590, 10862)
(2356, 10883) (590, 10883)
(2356, 10895) (590, 10895)
(2356, 10907) (590, 10907)
(2356, 10916) (590, 10916)
(2356, 10920) (590, 10920)
(2356, 10925) (590, 10925)
(2356, 10934) (590, 10934)
(2356, 11249) (590, 11249)
(2356, 11256) (590, 11256)
(2356, 11497) (590, 11497)
(2356, 11704) (590, 11704)
(2356, 11713) (590, 11713)
(2356, 11808) (590, 11808)
(2356, 11814) (590, 11814)
(2356, 11820) (590, 11820)
(2356, 11826) (590, 11826)
(2356, 11860) (590, 11860)
(2356, 11894) (590, 11894)
(2356, 11916) (590, 11916)
(2356, 11938) (590, 11938)
(2356, 11965) (590, 11965)
(2356, 11992) (590, 11992)
(2356, 12005) (590, 12005)
(2356, 12018) (590, 12018)
(2356, 12041)

In [190]:
x_train_df.shape

(2356, 18281)

In [191]:
len(element_remove)

28

In [192]:
''' Concat the nontext features with the x_train_df that we have got after doing one hot encoding of text features'''

def add_features(x_train,x_test,x_train_df,x_test_df,features):
  count = 0
  for feature in features:
    if count == 0:
      x_train_df = np.hstack((x_train_df.toarray(),x_train[feature].values.reshape(-1,1)))
      x_test_df = np.hstack((x_test_df.toarray(),x_test[feature].values.reshape(-1,1)))
    else:
      x_train_df = np.hstack((x_train_df,x_train[feature].values.reshape(-1,1)))
      x_test_df = np.hstack((x_test_df,x_test[feature].values.reshape(-1,1)))
    count +=1
  return x_train_df,x_test_df

In [193]:
x_tr,x_te=add_features(x_train_r,x_test_r,x_train_df,x_test_df,element_remove)

In [194]:
x_tr.shape,x_te.shape

((2356, 18309), (590, 18309))

# Model

In [204]:
neigh = RandomForestRegressor(max_depth=50,n_estimators=10,random_state=0)
neigh.fit(x_tr, y_train_reg)

y_train_pred = neigh.predict(x_tr)
y_test_pred = neigh.predict(x_te)

mse_tr=mean_squared_error(y_train_reg, y_train_pred,squared=False)
mse_te=mean_squared_error(y_test_reg, y_test_pred,squared=False)

In [210]:
print('RMSE for train ',round(mse_tr,4))
print('RMSE for test ',round(mse_te,4))

RMSE for train  7968.9434
RMSE for test  14735.7769
